# EDA 미니 프로젝트

### 전처리 목적
본 노트북은 영화 데이터에서 **관람인원수(VIEWNG_NMPR_CO)** 를 핵심 지표로 활용하기 위해, 날짜(OPN_DE) 및 관람인원수(VIEWNG_NMPR_CO)의 형식 불일치/결측/비정상 값을 정제하고 범주형 변수(DISTB_CMPNY_NM, GENRE_NM, NLTY_NM)를 분석 가능한 형태로 표준화 하는 과정을 일관된 정책으로 수행한다.

### 전처리 설계 원칙
- 안전성 우선: 원본 컬럼은 최대한 보존하고, 정제 컬럼을 별도로 생성 후 검증 뒤 반영한다.
- 정책 기반 처리: “왜 제거/대체/표준화 했는지”를 단계별로 명시해 재현성과 설명가능성을 확보한다.
- 검증 루프 포함: 정제 전/후에 비정상 값 개수와 샘플을 확인하여 품질을 보장한다.

## 1) 기본 설정 세팅

In [13]:
# (선택) 라이브러리 설치가 안 될 때만 실행
# !pip install pandas matplotlib seaborn

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

한글 폰트 설정 (Windows)

In [15]:
import matplotlib
import matplotlib.font_manager as fm

# Windows 기본 폰트 경로 예시 (환경에 따라 폰트 파일명은 다를 수 있음)
font_path = "C:\\Windows\\Fonts\\H2GTRM.TTF"
font_prop = fm.FontProperties(fname=font_path)
font_name = font_prop.get_name()

matplotlib.rc('font', family=font_name)
plt.rc('axes', unicode_minus=False)  # 음수 기호 깨짐 방지

## 2) 데이터셋 로드

In [16]:
movie_df = pd.read_csv("./data/merged_56_utf8.csv")
movie_df.head()

,NO,MOVIE_NM,DRCTR_NM,MAKR_NM,INCME_CMPNY_NM,DISTB_CMPNY_NM,OPN_DE,MOVIE_TY_NM,MOVIE_STLE_NM,NLTY_NM,TOT_SCRN_CO,SALES_PRICE,VIEWNG_NMPR_CO,SEOUL_SALES_PRICE,SEOUL_VIEWNG_NMPR_CO,GENRE_NM,GRAD_NM,MOVIE_SDIV_NM
0,1.0,남산의 부장들,우민호,(주)하이브미디어코프,NaN,(주)쇼박스,20200122,개봉영화,장편,한국,1659,41223596650,4750104,9851448590,1113402,드라마,15세이상관람가,일반영화
1,2.0,다만 악에서 구하소서,홍원찬,(주)하이브미디어코프,NaN,(주)씨제이이엔엠,20200805,개봉영화,장편,한국,1998,38554527990,4352669,8906277620,984042,범죄,15세이상관람가,일반영화
2,3.0,반도,연상호,(주)영화사레드피터,NaN,(주)넥스트엔터테인먼트월드(NEW),20200715,개봉영화,장편,한국,2575,33071341280,3812080,8005831840,895163,액션,15세이상관람가,일반영화
3,4.0,히트맨,최원섭,베리굿스튜디오(주),NaN,롯데컬처웍스(주)롯데엔터테인먼트,20200122,개봉영화,장편,한국,1122,20614278000,2406232,4072414720,472840,코미디,15세이상관람가,일반영화
4,5.0,테넷,크리스토퍼 놀란,NaN,워너브러더스 코리아(주),워너브러더스 코리아(주),20200826,개봉영화,장편,미국,2228,18396929850,1992214,6615682400,677442,액션,12세이상관람가,일반영화


In [17]:
movie_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 12472 entries, 0 to 12471
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   NO                    11877 non-null  float64
 1   MOVIE_NM              11877 non-null  str    
 2   DRCTR_NM              10125 non-null  str    
 3   MAKR_NM               5023 non-null   str    
 4   INCME_CMPNY_NM        4651 non-null   str    
 5   DISTB_CMPNY_NM        11876 non-null  str    
 6   OPN_DE                11875 non-null  str    
 7   MOVIE_TY_NM           11877 non-null  str    
 8   MOVIE_STLE_NM         11877 non-null  str    
 9   NLTY_NM               11877 non-null  str    
 10  TOT_SCRN_CO           11472 non-null  str    
 11  SALES_PRICE           5382 non-null   str    
 12  VIEWNG_NMPR_CO        10019 non-null  str    
 13  SEOUL_SALES_PRICE     8229 non-null   str    
 14  SEOUL_VIEWNG_NMPR_CO  10380 non-null  str    
 15  GENRE_NM              11842 no

In [18]:
movie_df.describe(include="all")

,NO,MOVIE_NM,DRCTR_NM,MAKR_NM,INCME_CMPNY_NM,DISTB_CMPNY_NM,OPN_DE,MOVIE_TY_NM,MOVIE_STLE_NM,NLTY_NM,TOT_SCRN_CO,SALES_PRICE,VIEWNG_NMPR_CO,SEOUL_SALES_PRICE,SEOUL_VIEWNG_NMPR_CO,GENRE_NM,GRAD_NM,MOVIE_SDIV_NM
count,11877.000000,11877,10125,5023,4651,11876,11875,11877,11877,11877,11472,5382,10019,8229,10380,11842,11877,11877
unique,NaN,7763,2823,935,249,639,1165,1,4,57,1057,3504,3459,3229,3037,21,7,2
top,NaN,더 퍼스트 슬램덩크,버드맨 텟페이,(주)영화사가을,유니버설픽쳐스인터내셔널 코리아(유),(주)영진크리에이티브,2021-08-,개봉영화,장편,한국,1.0,6000,1.0,0.0,0.0,드라마,청소년관람불가,일반영화
freq,NaN,23,162,426,232,1107,321,11877,11832,5062,2467,914,1915,2919,2919,2573,4751,7768
mean,251.233224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,411.393013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,54.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,109.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,195.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3) 데이터 전처리 1단계: 중복 제거 → 필요한 컬럼만 추출
**기준/이유**
- 중복 제거는 컬럼 축소보다 먼저 수행한다.
    - 컬럼을 먼저 줄이면 “원래는 다른 행인데, 남은 컬럼만 보면 동일해 보이는” 케이스가 발생할 수 있어 중복 판정이 왜곡될 수 있다.
- 이후 분석에 필요한 컬럼만 선택해 데이터 크기/복잡도를 줄인다.

### 4-1) 중복 여부 확인 및 제거

In [19]:
dup_mask = movie_df.duplicated(keep=False)
movie_df.loc[dup_mask].head()

,NO,MOVIE_NM,DRCTR_NM,MAKR_NM,INCME_CMPNY_NM,DISTB_CMPNY_NM,OPN_DE,MOVIE_TY_NM,MOVIE_STLE_NM,NLTY_NM,TOT_SCRN_CO,SALES_PRICE,VIEWNG_NMPR_CO,SEOUL_SALES_PRICE,SEOUL_VIEWNG_NMPR_CO,GENRE_NM,GRAD_NM,MOVIE_SDIV_NM
3400,1.0,모가디슈,류승완,"(주)덱스터스튜디오,(주)외유내강,(주)필름케이",NaN,롯데컬처웍스(주)롯데엔터테인먼트,2021-07-,개봉영화,장편,한국,NaN,NaN,NaN,NaN,NaN,액션,15세이상관람가,일반영화
3404,5.0,보스 베이비 2,톰 맥그라스,NaN,유니버설픽쳐스인터내셔널 코리아(유),유니버설픽쳐스인터내셔널 코리아(유),2021-07-,개봉영화,장편,미국,NaN,NaN,NaN,NaN,NaN,애니메이션,전체관람가,일반영화
3446,47.0,나는 나대로 혼자서 간다,오키타 슈이치,NaN,(주)영화사 진진,(주)영화사 진진,2021-07-,개봉영화,장편,일본,23.0,NaN,NaN,NaN,NaN,드라마,12세이상관람가,독립/예술영화
3453,54.0,더 퍼지: 포에버,에베라도 발레리오 구트,NaN,유니버설픽쳐스인터내셔널 코리아(유),유니버설픽쳐스인터내셔널 코리아(유),2021-07-,개봉영화,장편,미국,52.0,NaN,NaN,NaN,NaN,스릴러,청소년관람불가,일반영화
3745,1.0,모가디슈,류승완,"(주)덱스터스튜디오,(주)외유내강,(주)필름케이",NaN,롯데컬처웍스(주)롯데엔터테인먼트,2021-07-,개봉영화,장편,한국,NaN,NaN,NaN,NaN,NaN,액션,15세이상관람가,일반영화


In [20]:
movie_df = movie_df.drop_duplicates(keep="first")

### 4-2) 필요한 컬럼만 선택
선택 기준
- 타깃(관람인원수): VIEWNG_NMPR_CO
- 설명변수(범주): DISTB_CMPNY_NM(배급사), GENRE_NM(장르), NLTY_NM(국적)
- 시간변수(개봉일): OPN_DE

In [21]:
use_cols = ["VIEWNG_NMPR_CO", "DISTB_CMPNY_NM", "GENRE_NM", "NLTY_NM", "OPN_DE"]
movie_df = movie_df[use_cols].copy()
movie_df.head()

,VIEWNG_NMPR_CO,DISTB_CMPNY_NM,GENRE_NM,NLTY_NM,OPN_DE
0,4750104,(주)쇼박스,드라마,한국,20200122
1,4352669,(주)씨제이이엔엠,범죄,한국,20200805
2,3812080,(주)넥스트엔터테인먼트월드(NEW),액션,한국,20200715
3,2406232,롯데컬처웍스(주)롯데엔터테인먼트,코미디,한국,20200122
4,1992214,워너브러더스 코리아(주),액션,미국,20200826


In [22]:
movie_df.info()

<class 'pandas.DataFrame'>
Index: 11855 entries, 0 to 12471
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   VIEWNG_NMPR_CO  10019 non-null  str  
 1   DISTB_CMPNY_NM  11853 non-null  str  
 2   GENRE_NM        11819 non-null  str  
 3   NLTY_NM         11854 non-null  str  
 4   OPN_DE          11852 non-null  str  
dtypes: str(5)
memory usage: 555.7 KB


## 4) 데이터 전처리 2단계: 컬럼별 정리
본 단계에서는 전처리 대상 컬럼을 (1) 핵심 수치형 2개와 (2) 범주형 3개로 구분하고, 각 컬럼별로 데이터 품질 이슈 → 처리 정책 → 기대 효과를 명시한다.
목표는 “분석/시각화/모델링에 바로 투입 가능한 형태”로 형식 표준화 + 결측/비정상 관리 + 범주 축약을 일관되게 수행하는 것이다.

### OPN_DE 컬럼

#### 컬럼 이슈 진단
**목표**

OPN_DE를 **분석 가능한 날짜형(datetime)** 으로 변환하기 위한 사전 점검.

관측된 이슈 유형(예시)

- YYYYMMDD.0 형태: 로딩 과정에서 float 흔적이 문자열로 남은 케이스
- NaN: 개봉일 정보 누락
- YYYY-MM- 형태: 연-월까지만 있고 일(day)이 비어있는 케이스
- 기타: 하이픈/공백/문자 혼입 등 비정형 케이스

1. 결측치(NaN) 개수 확인

In [23]:
nan_count = movie_df["OPN_DE"].isna().sum()
print("NaN count:", nan_count)

NaN count: 3


2. 8자리(YYYYMMDD)가 아닌 값 찾기
- 정책상 최종 목표 포맷은 YYYYMMDD(8자리 숫자)로 정규화
- NaN 포함 시 문자열 함수가 예외/의도치 않은 결과를 만들 수 있으므로 astype(str)로 안전 처리 후 점검

In [24]:
mask_not_yyyymmdd = ~movie_df["OPN_DE"].astype(str).str.match(r"^\d{8}$")
movie_df.loc[mask_not_yyyymmdd, "OPN_DE"].head(10)

3182    20210526.0
3183    20210603.0
3184    20210616.0
3185    20210519.0
3186    20210623.0
3187    20210623.0
3188    20210617.0
3189    20210609.0
3190    20210617.0
3191    20210127.0
Name: OPN_DE, dtype: str

In [25]:
print("Not YYYYMMDD count:", mask_not_yyyymmdd.sum())

Not YYYYMMDD count: 5269


3. .0가 포함된 값 확인

In [26]:
mask_dot0 = movie_df["OPN_DE"].astype(str).str.contains(r".0", regex=True)
movie_df.loc[mask_dot0, "OPN_DE"].head(20)

print(".0 포함 count:", mask_dot0.sum())

.0 포함 count: 11771


#### 정제 함수 정의
**처리 정책(Policy)**
1. OPN_DE 결측치 행은 제거
- 기준: 개봉일은 시계열/연도별 집계에서 핵심 키이며, 결측이 소수인 경우(예: 3건 수준) 데이터 손실 영향이 작다.
2. YYYYMMDD.0 → YYYYMMDD로 정리
3. YYYY-MM- → 날짜(day)가 없으므로 **해당 월의 1일(01)**을 부여하여 YYYYMM01로 정규화
- 기준: “월 단위 정보라도 살릴 가치가 있다”는 보수적 보정. (필요 시 이후 단계에서 월 단위 분석 가능)
4. 그 외 비정형 입력은 숫자만 추출 후 길이에 따라 day를 보정(방어적 처리)
- YYYYMM(6자리) → YYYYMM01
- YYYY(4자리) → YYYY0101
5. 최종적으로 8자리만 유지(과도한 길이/잡문자 유입 방지)
**반환**
- OPN_DE를 **문자열 'YYYYMMDD'**로 정제한 DataFrame

In [27]:
def clean_opn_de_to_datetime(df: pd.DataFrame, col: str = "OPN_DE") -> pd.DataFrame:
    """
    개봉일자(OPN_DE)를 정규화한 뒤 datetime64[ns]로 변환한다.

    처리 정책
    - 결측치 제거(dropna)
    - 'YYYY-MM-' -> 'YYYY-MM-01'로 보정
    - 'YYYYMMDD.0' -> '.0' 제거
    - 숫자만 남기고 길이 보정:
        * YYYYMM(6) -> YYYYMM01
        * YYYY(4)  -> YYYY0101
      이후 8자리로 절단
    - pd.to_datetime(format='%Y%m%d', errors='coerce')로 변환
    - 변환 실패(NaT) 제거

    Returns
    - col: datetime64[ns]
    """
    df = df.copy()

    # 1) 결측치 제거
    df = df.dropna(subset=[col])

    # 2) 문자열 정리
    s = df[col].astype(str).str.strip()

    # 3) 'YYYY-MM-' -> 'YYYY-MM-01'
    s = s.str.replace(r"^(\d{4})-(\d{2})-$", r"\1-\2-01", regex=True)

    # 4) '.0' 제거
    s = s.str.replace(r"\.0$", "", regex=True)

    # 5) 숫자만 남기기
    s = s.str.replace(r"\D", "", regex=True)

    # 6) 길이 보정(방어적)
    mask_6 = s.str.len().eq(6)
    s.loc[mask_6] = s.loc[mask_6] + "01"

    mask_4 = s.str.len().eq(4)
    s.loc[mask_4] = s.loc[mask_4] + "0101"

    # 7) 8자리로 제한
    s = s.str[:8]

    # 8) datetime 변환
    dt = pd.to_datetime(s, format="%Y%m%d", errors="coerce")
    df[col] = dt

    # 9) NaT 제거
    df = df.dropna(subset=[col])

    return df

In [28]:
movie_df = clean_opn_de_to_datetime(movie_df, col="OPN_DE")
movie_df["OPN_DE"].head(10)

0   2020-01-22
1   2020-08-05
2   2020-07-15
3   2020-01-22
4   2020-08-26
5   2019-12-19
6   2020-06-24
7   2020-07-29
8   2020-09-29
9   2020-01-08
Name: OPN_DE, dtype: datetime64[us]

In [29]:
movie_df.info()

<class 'pandas.DataFrame'>
Index: 11752 entries, 0 to 12471
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   VIEWNG_NMPR_CO  9917 non-null   str           
 1   DISTB_CMPNY_NM  11751 non-null  str           
 2   GENRE_NM        11717 non-null  str           
 3   NLTY_NM         11752 non-null  str           
 4   OPN_DE          11752 non-null  datetime64[us]
dtypes: datetime64[us](1), str(4)
memory usage: 550.9 KB


#### 최종 검증 및 정리
**기준/이유**
- datetime 변환 후에도 NaT가 남아있다면 “정제 정책으로 커버 못한 입력”이 존재하는 것이므로 샘플을 확인한다.
- 분석에서 날짜가 필요한 경우, NaT는 제거하여 시간축 기반 분석의 일관성을 확보한다.

In [30]:
print("NaT count after conversion:", movie_df["OPN_DE"].isna().sum())

NaT count after conversion: 0


In [31]:
movie_df.loc[movie_df["OPN_DE"].isna(), ["OPN_DE"]].head()

,OPN_DE


In [32]:
movie_df.info()

<class 'pandas.DataFrame'>
Index: 11752 entries, 0 to 12471
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   VIEWNG_NMPR_CO  9917 non-null   str           
 1   DISTB_CMPNY_NM  11751 non-null  str           
 2   GENRE_NM        11717 non-null  str           
 3   NLTY_NM         11752 non-null  str           
 4   OPN_DE          11752 non-null  datetime64[us]
dtypes: datetime64[us](1), str(4)
memory usage: 550.9 KB


### VIEWNG_NMPR_CO 컬럼

#### 결측치 처리
**기준/이유**
- 본 프로젝트의 핵심 지표는 관람인원수이므로, 값이 없는 행은 모델링/집계/시각화에서 활용 불가하다.
- 따라서 VIEWNG_NMPR_CO가 결측인 행은 제거한다.

In [33]:
movie_df = movie_df.dropna(subset=["VIEWNG_NMPR_CO"])
movie_df.info()

<class 'pandas.DataFrame'>
Index: 9917 entries, 0 to 12471
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   VIEWNG_NMPR_CO  9917 non-null   str           
 1   DISTB_CMPNY_NM  9916 non-null   str           
 2   GENRE_NM        9886 non-null   str           
 3   NLTY_NM         9917 non-null   str           
 4   OPN_DE          9917 non-null   datetime64[us]
dtypes: datetime64[us](1), str(4)
memory usage: 464.9 KB


#### 컬럼 이슈 진단
**관측된 이슈**
- 문자열 타입으로 들어와 있음
- 값이 다음 패턴으로 혼재할 수 있음
    - 정수 문자열: "1234"
    - 소수점 흔적: "1234.0"
    - 콤마 포함: "1,234"

**목표**
전처리 후 VIEWNG_NMPR_CO를 **정수형(int64)**으로 변환하여 수치 분석 가능하게 만든다.

정수만 있는 경우가 아니라면 표시 (콤마/소수점/기타 문자 포함 탐지)

In [34]:
mask_not_int = ~movie_df["VIEWNG_NMPR_CO"].astype(str).str.match(r"^\d+$")
movie_df.loc[mask_not_int, "VIEWNG_NMPR_CO"].head(20)

3244    975.0
3245    955.0
3246    946.0
3247    919.0
3248    878.0
3249    694.0
3250    692.0
3251    640.0
3252    639.0
3253    509.0
3254    500.0
3255    468.0
3256    467.0
3257    466.0
3258    416.0
3259    410.0
3260    368.0
3261    360.0
3262    310.0
3263    309.0
Name: VIEWNG_NMPR_CO, dtype: str

In [35]:
print("Not pure integer string count:", mask_not_int.sum())

Not pure integer string count: 6002


허용 패턴: '1234' 또는 '1234.0' (콤마는 제거 후 처리 예정)

In [36]:
mask_not_valid = ~movie_df["VIEWNG_NMPR_CO"].astype(str).str.replace(",", "", regex=False).str.match(r"^\d+(.0)?$")
movie_df.loc[mask_not_valid, "VIEWNG_NMPR_CO"].head(20)

Series([], Name: VIEWNG_NMPR_CO, dtype: str)

#### 정제 함수 정의
**처리 정책(Policy)**
1. 콤마 제거 후 문자열 통일
2. pd.to_numeric(..., errors='coerce')로 숫자 변환
    - 변환 실패 값은 NaN으로 표준화 (이후 제거)
3. 소수점 흔적(.0)이 포함된 값도 숫자로 흡수한 뒤 정수형으로 캐스팅
4. 최종적으로 결측/비정상 값은 제거하여 분석 품질 확보

주의: 정수형 변환은 astype("int64")로 고정하는 것이 가장 명확하다.

In [37]:
def clean_viewng_nmpr_co_to_int(df: pd.DataFrame, col: str = "VIEWNG_NMPR_CO") -> pd.DataFrame:
    """
    관람인원수(VIEWNG_NMPR_CO)를 정수형(int64)으로 정제한다.

    처리 정책
    - 결측치 제거(dropna)
    - 콤마 제거 후 숫자 변환(pd.to_numeric, errors='coerce')
    - 변환 실패는 NaN 처리 후 제거
    - 최종 int64로 캐스팅

    Returns
    - col: int64
    """
    df = df.copy()

    # 1) 결측치 제거
    df = df.dropna(subset=[col])

    # 2) 문자열 통일 + 콤마 제거
    s = df[col].astype(str).str.replace(",", "", regex=False).str.strip()

    # 3) 숫자 변환 (불가 -> NaN)
    num = pd.to_numeric(s, errors="coerce")
    df[col] = num

    # 4) 변환 실패 제거
    df = df.dropna(subset=[col])

    # 5) 정수형 변환
    df[col] = df[col].astype("int64")

    return df

In [38]:
movie_df = clean_viewng_nmpr_co_to_int(movie_df, col="VIEWNG_NMPR_CO")

In [39]:
print(movie_df["VIEWNG_NMPR_CO"].dtype)
movie_df["VIEWNG_NMPR_CO"].head()

int64


0    4750104
1    4352669
2    3812080
3    2406232
4    1992214
Name: VIEWNG_NMPR_CO, dtype: int64

#### 최종 검증 및 정리
**기준/이유**
- 전처리 후에도 NaN이 남아있지 않은지
- 0 값이 과도하게 존재하지 않는지(수집 오류/의미 없는 레코드 가능성)를 확인해 데이터 품질 리스크를 사전에 식별한다.

In [40]:
null_count = movie_df["VIEWNG_NMPR_CO"].isna().sum()
zero_count = (movie_df["VIEWNG_NMPR_CO"] == 0).sum()

print(f"1) NaN(결측치) 개수: {null_count}개")
print(f"2) 0 값 데이터 개수 : {zero_count}개")
print(f"3) 전체 데이터 개수 : {len(movie_df)}개")

1) NaN(결측치) 개수: 0개
2) 0 값 데이터 개수 : 0개
3) 전체 데이터 개수 : 9917개


### 범주형 변수 정리 개요
이제 남은 컬럼은 범주형(카테고리) 변수 3개이며, 목표는 다음과 같다.
- DISTB_CMPNY_NM(배급사): 표기 흔들림/공동배급/법인수식어 등을 정리하여 표준 배급사명으로 통합
- GENRE_NM(장르): 결측은 분석 누락 방지를 위해 **'미상'**으로 통일
- NLTY_NM(국적): 필요 시 '한국/미국/일본/기타' 등으로 top-N + 기타 전략 적용 가능

### DISTB_CMPNY_NM 컬럼

#### 컬럼 이슈 진단
**기준/이유**
- 배급사는 표기 변형이 많다(띄어쓰기/법인명/영문/한글/계열사).
- 먼저 빈도 상위와 키워드 매칭을 통해 “통합 규칙이 필요한 집단”을 파악한다.

In [41]:
print("--- 빈도수 상위 30개 배급사 명칭 ---")
print(movie_df["DISTB_CMPNY_NM"].value_counts().head(30))

keywords = ["씨제이", "CJ", "롯데", "넥스트", "NEW", "쇼박스", "플러스엠", "워너", "디즈니", "유니버설"]

print("\n--- 주요 키워드 포함 명칭 리스트(샘플) ---")
for kw in keywords:
    matches = movie_df[movie_df["DISTB_CMPNY_NM"].astype(str).str.contains(kw, na=False, case=False)][
        "DISTB_CMPNY_NM"].unique()
print(f"[{kw}] 관련 명칭들 ({len(matches)}종류) 샘플:")
print(matches[:5])
print("-" * 40)

--- 빈도수 상위 30개 배급사 명칭 ---
DISTB_CMPNY_NM
(주)영진크리에이티브            1107
(주)영화사가을                427
(주)라온컴퍼니플러스             270
(주)가온콘텐츠                237
(주)샤이커뮤니케이션즈            221
(주)도키엔터테인먼트             191
(주)씨맥스커뮤니케이션즈           162
(유)조이앤시네마               149
유니버설픽쳐스인터내셔널 코리아(유)     129
(주)코빈커뮤니케이션즈            124
(주)넥스트엔터테인먼트월드(NEW)     119
(주)영화사 진진               119
씨제이 씨지브이(CJ CGV)(주)     119
월트디즈니컴퍼니코리아 유한책임회사      116
롯데컬처웍스(주)롯데엔터테인먼트       115
(주)픽쳐레스크                111
(주)디스테이션                109
(주)엣나인필름                101
케이엘 픽쳐스                 101
주식회사 루믹스미디어             101
스마일컨텐츠                   99
(주)씨제이이엔엠                98
워너브러더스 코리아(주)            95
(주)트리플픽쳐스                93
와이드 릴리즈(주)               92
(주)영화사히트                 91
(주)팝엔터테인먼트               89
(주)미디어캐슬                 89
(주)드림팩트엔터테인먼트            87
판씨네마(주)                  86
Name: count, dtype: int64

--- 주요 키워드 포함 명칭 리스트(샘플) ---
[유니버설] 관련 명칭들 (1종류) 샘플:
<StringArray>
['유니버설픽쳐스인터내셔널 코리아(유)']
L

#### 결측치 처리
**기준/이유**
- NaN을 그대로 두면 장르별 집계/시각화에서 자동 제외되어 분석 대상이 의도치 않게 줄어든다.
- NaN 뿐 아니라 '', ' ' 같은 빈 문자열도 실질적으로 결측이다.
- 범주형 전처리 전에 결측 규모를 파악하고, 정책(제거 vs 대체)을 결정한다.
- 따라서 결측치는 **'미상'** 으로 대체하여 분석 과정에서 하나의 명시적 범주로 취급한다.

In [42]:
nan_count = movie_df["DISTB_CMPNY_NM"].isna().sum()
empty_str_count = (movie_df["DISTB_CMPNY_NM"].astype(str).str.strip() == "").sum()

print("NaN count:", nan_count)
print("Empty/blank string count:", empty_str_count)

NaN count: 1
Empty/blank string count: 0


In [43]:
movie_df

,VIEWNG_NMPR_CO,DISTB_CMPNY_NM,GENRE_NM,NLTY_NM,OPN_DE
0,4750104,(주)쇼박스,드라마,한국,2020-01-22
1,4352669,(주)씨제이이엔엠,범죄,한국,2020-08-05
2,3812080,(주)넥스트엔터테인먼트월드(NEW),액션,한국,2020-07-15
3,2406232,롯데컬처웍스(주)롯데엔터테인먼트,코미디,한국,2020-01-22
4,1992214,워너브러더스 코리아(주),액션,미국,2020-08-26
...,...,...,...,...,...
12467,445,메가박스중앙(주) 플러스엠 엔터테인먼트,범죄,한국,2025-04-16
12468,430,(주)박수엔터테인먼트,애니메이션,미국,2025-04-10
12469,424,(주)넥스트엔터테인먼트월드(NEW),드라마,일본,2023-11-29
12470,405,씨제이 씨지브이(CJ CGV)(주),애니메이션,한국,2025-01-01


In [44]:
movie_df['DISTB_CMPNY_NM'] = movie_df['DISTB_CMPNY_NM'].fillna('미상')

#### 배급사 1차 정제: 공동배급 분리 + 법인 수식어 제거
**처리 정책(Policy)**
1. 결측은 일단 빈 문자열로 채운 뒤 처리 (문자열 메서드 안정성)
2. 공동 배급 표기(콤마 연결)는 첫 번째 배급사를 대표값으로 사용
    - 기준: 단순화 목적(EDA)이며, 공동배급을 모두 반영하려면 별도 파싱/다중 라벨링이 필요
3. (주), 주식회사 등 법인 수식어 및 (NEW)/NEW 등의 불필요 토큰 제거
4. 결과는 DIST_CLEAN 정제 컬럼으로 생성 (원본 보존)

In [45]:
def clean_distb_cmpny_nm(df: pd.DataFrame, col: str = "DISTB_CMPNY_NM") -> pd.DataFrame:
    """
    배급사명(DISTB_CMPNY_NM)을 1차 정제하여 'DIST_CLEAN' 컬럼을 생성한다.

    처리 정책
    - NaN -> ''로 치환(문자열 처리 안정성)
    - 콤마로 연결된 공동배급 표기에서 첫 번째 값만 대표값으로 사용
    - 법인 수식어 제거: (주), 주식회사, (유), 유한책임회사
    - 불필요 토큰 제거: (NEW), NEW
    - strip()으로 공백 정리

    Returns
    - DIST_CLEAN: 정제된 배급사명(문자열)
    """
    df = df.copy()

    s = df[col].fillna("").astype(str)

    # 공동배급: 첫 번째만 사용
    s = s.str.split(",").str[0]

    # 토큰 제거 및 공백 정리
    s = (
        s.str.replace(r"\(NEW\)|NEW", "", regex=True, case=False)
         .str.replace(r"\(주\)|주식회사|\(유\)|유한책임회사", "", regex=True)
         .str.strip()
    )

    df["DIST_CLEAN"] = s
    return df

In [46]:
movie_df = clean_distb_cmpny_nm(movie_df)

In [47]:
movie_df["DIST_CLEAN"].unique()[:20]

array(['쇼박스', '씨제이이엔엠', '넥스트엔터테인먼트월드', '롯데컬처웍스롯데엔터테인먼트', '워너브러더스 코리아',
       '유니버설픽쳐스인터내셔널 코리아', '메가박스중앙플러스엠', '에이스메이커무비웍스',
       '소니픽쳐스엔터테인먼트코리아극장배급지점', '스마일이엔티', '누리픽쳐스', '월트디즈니컴퍼니코리아', '리틀빅픽쳐스',
       'CGV아트하우스', '제이앤씨미디어그룹', '올스타엔터테인먼트', '이놀미디어', '키위미디어그룹', '홈초이스',
       '영화사 그램'], dtype=object)

#### 배급사 표준화(통합) 함수: 그룹 단위 매핑
**기준/이유**
- EDA/시각화에서 배급사 카테고리가 과도하게 분산되면 인사이트 도출이 어려워진다.
- 따라서 대표 그룹(예: CJ ENM/워너/디즈니 등)은 규칙 기반으로 표준 라벨로 통합한다.
- 결과는 DIST_MERGED 컬럼에 반영한다.

In [48]:
def merge_dist_name(name) -> str:
    """
    정제된 배급사명(DIST_CLEAN)을 표준 라벨로 통합한다(규칙 기반).

    Parameters
    - name: 단일 배급사명(문자열/NaN 가능)

    Returns
    - 표준화된 배급사명(문자열)
    """
    if pd.isna(name) or not str(name).strip():
        return name

    s = str(name).strip()
    s = " ".join(s.split())  # 연속 공백 정리

    # CJ ENM
    if ("씨제이이엔엠" in s) or ("CJ ENM" in s) or ("씨제이" in s and "이엔엠" in s):
        return "CJ ENM"

    # CJ CGV
    if ("CGV" in s) or ("씨지브이" in s):
        return "CJ CGV"

    # CJ 4DPLEX
    if ("포디플렉스" in s) or ("4DPLEX" in s):
        return "CJ 4DPLEX"

    # 롯데
    if "롯데컬처웍스" in s:
        return "롯데컬처웍스"

    # 플러스엠
    if ("메가박스중앙" in s) or ("플러스엠" in s):
        return "플러스엠"

    # 디즈니 / 유니버설 / 소니 / 워너
    if ("월트디즈니" in s) or ("디즈니컴퍼니코리아" in s):
        return "월트디즈니"

    if "유니버설픽쳐스" in s:
        return "유니버설픽쳐스"

    if "소니픽쳐스" in s:
        return "소니픽쳐스"

    if "워너브러더스" in s:
        return "워너브러더스"

    # 띄어쓰기 차이 방어
    compact = s.replace(" ", "")

    if compact in ["영화사빅", "영화사진진"]:
        return compact

    if "더쿱" in compact:
        return "더쿱"
    if "무비다이브" in compact:
        return "무비다이브"
    if "모쿠슈라" in compact:
        return "모쿠슈라"

    return s

In [49]:
movie_df["DIST_MERGED"] = movie_df["DIST_CLEAN"].apply(merge_dist_name)
movie_df[["DISTB_CMPNY_NM", "DIST_CLEAN", "DIST_MERGED"]].head()

,DISTB_CMPNY_NM,DIST_CLEAN,DIST_MERGED
0,(주)쇼박스,쇼박스,쇼박스
1,(주)씨제이이엔엠,씨제이이엔엠,CJ ENM
2,(주)넥스트엔터테인먼트월드(NEW),넥스트엔터테인먼트월드,넥스트엔터테인먼트월드
3,롯데컬처웍스(주)롯데엔터테인먼트,롯데컬처웍스롯데엔터테인먼트,롯데컬처웍스
4,워너브러더스 코리아(주),워너브러더스 코리아,워너브러더스


#### 배급사 top-N 전략: 상위 30 + 기타(Other)
**기준/이유**
- 배급사 종류가 너무 많으면 시각화/비교가 어려워진다.
- 따라서 상위 30개 배급사는 유지하고, 나머지는 **'기타'**로 묶어 EDA의 가독성을 높인다.
- 결과 컬럼: DIST_TOP30

In [50]:
dist_count = movie_df["DIST_MERGED"].value_counts()
top30_dist = dist_count.head(30).index.tolist()
print(top30_dist)

['영진크리에이티브', '영화사가을', '라온컴퍼니플러스', '가온콘텐츠', '샤이커뮤니케이션즈', 'CJ CGV', '도키엔터테인먼트', '씨맥스커뮤니케이션즈', '조이앤시네마', '롯데컬처웍스', '영화사진진', '넥스트엔터테인먼트월드', '유니버설픽쳐스', '코빈커뮤니케이션즈', '플러스엠', '월트디즈니', '드림팩트엔터테인먼트', '디스테이션', '와이드 릴리즈', '엣나인필름', '픽쳐레스크', '찬란', '케이엘 픽쳐스', '루믹스미디어', 'CJ ENM', '스마일컨텐츠', '워너브러더스', '제이앤씨미디어그룹', '트리플픽쳐스', '팝엔터테인먼트']


In [51]:
movie_df["DIST_TOP30"] = movie_df["DIST_MERGED"].apply(lambda x: x if x in top30_dist else "기타")
movie_df["DIST_TOP30"].value_counts()

DIST_TOP30
기타             4618
영진크리에이티브       1108
영화사가을           427
라온컴퍼니플러스        270
가온콘텐츠           237
샤이커뮤니케이션즈       221
CJ CGV          193
도키엔터테인먼트        191
씨맥스커뮤니케이션즈      162
조이앤시네마          150
롯데컬처웍스          141
영화사진진           137
넥스트엔터테인먼트월드     131
유니버설픽쳐스         129
코빈커뮤니케이션즈       124
플러스엠            116
월트디즈니           116
드림팩트엔터테인먼트      115
디스테이션           115
와이드 릴리즈         114
엣나인필름           111
픽쳐레스크           111
찬란              105
케이엘 픽쳐스         101
루믹스미디어          101
CJ ENM           99
스마일컨텐츠           99
워너브러더스           95
제이앤씨미디어그룹        95
트리플픽쳐스           93
팝엔터테인먼트          92
Name: count, dtype: int64

In [52]:
movie_df.info()

<class 'pandas.DataFrame'>
Index: 9917 entries, 0 to 12471
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   VIEWNG_NMPR_CO  9917 non-null   int64         
 1   DISTB_CMPNY_NM  9917 non-null   str           
 2   GENRE_NM        9886 non-null   str           
 3   NLTY_NM         9917 non-null   str           
 4   OPN_DE          9917 non-null   datetime64[us]
 5   DIST_CLEAN      9917 non-null   object        
 6   DIST_MERGED     9917 non-null   str           
 7   DIST_TOP30      9917 non-null   str           
dtypes: datetime64[us](1), int64(1), object(1), str(5)
memory usage: 697.3+ KB


### GENRE_NM 컬럼

#### 결측치 처리
**처리 기준(Policy)**
- GENRE_NM 결측은 “장르가 없다”가 아니라 “장르 정보가 누락/미제공”일 가능성이 크다.
- NaN을 그대로 두면 장르별 집계/시각화에서 자동 제외되어 분석 대상이 의도치 않게 줄어든다.
- 따라서 결측치는 **'미상'** 으로 대체하여 분석 과정에서 하나의 명시적 범주로 취급한다.

In [53]:
movie_df['GENRE_NM'].isna().sum()

np.int64(31)

In [54]:
movie_df["GENRE_NM"] = movie_df["GENRE_NM"].fillna("미상")
movie_df["GENRE_NM"].value_counts().head(30)

GENRE_NM
멜로/로맨스      2263
드라마         2079
성인물(에로)     1703
애니메이션        829
다큐멘터리        596
액션           588
코미디          285
공연           284
공포(호러)       263
스릴러          175
범죄           147
기타           138
미스터리         106
SF            92
판타지           90
전쟁            64
어드벤처          58
가족            55
뮤지컬           41
미상            31
사극            23
서부극(웨스턴)       7
Name: count, dtype: int64

In [55]:
movie_df.info()

<class 'pandas.DataFrame'>
Index: 9917 entries, 0 to 12471
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   VIEWNG_NMPR_CO  9917 non-null   int64         
 1   DISTB_CMPNY_NM  9917 non-null   str           
 2   GENRE_NM        9917 non-null   str           
 3   NLTY_NM         9917 non-null   str           
 4   OPN_DE          9917 non-null   datetime64[us]
 5   DIST_CLEAN      9917 non-null   object        
 6   DIST_MERGED     9917 non-null   str           
 7   DIST_TOP30      9917 non-null   str           
dtypes: datetime64[us](1), int64(1), object(1), str(5)
memory usage: 697.3+ KB


### NLTY_NM 컬럼

#### 기본 점검

In [56]:
movie_df["NLTY_NM"].isna().sum()

np.int64(0)

In [57]:
movie_df["NLTY_NM"].unique()

<StringArray>
[      '한국',       '미국',       '영국',      '프랑스',       '일본',       '대만',
       '중국',       '홍콩',      '벨기에',      '스페인',      '러시아',    '우크라이나',
     '이탈리아',       '기타',       '독일',       '호주',      '캐나다',     '네덜란드',
     '아일랜드',      '폴란드',     '노르웨이',      '브라질',      '헝가리',      '덴마크',
      '핀란드',      '스위스',    '슬로바키아',     '불가리아',       '태국',      '스웨덴',
       '터키',      '필리핀',      '멕시코',    '크로아티아',   '우즈베키스탄',     '포르투갈',
    '오스트리아',      '모로코',    '아르헨티나',     '이스라엘',     '뉴질랜드', '남아프리카공화국',
     '우루과이',      '베트남',    '카자흐스탄',    '에스토니아',      '이집트',    '아이슬란드',
    '인도네시아',       '인도',       '체코',       '몽고',       '이란',    '말레이시아']
Length: 54, dtype: str

In [58]:
movie_df["NLTY_NM"].value_counts()

NLTY_NM
한국          4289
일본          2749
미국          1258
프랑스          272
영국           239
중국           173
기타           129
독일           118
이탈리아         110
캐나다           87
러시아           76
스페인           70
홍콩            62
호주            44
헝가리           25
대만            24
노르웨이          21
네덜란드          16
폴란드           15
핀란드           14
아일랜드          13
덴마크           12
벨기에           10
우크라이나          7
브라질            7
베트남            7
인도             7
멕시코            5
오스트리아          5
태국             4
필리핀            4
포르투갈           4
모로코            4
아르헨티나          4
이스라엘           4
스위스            3
터키             3
카자흐스탄          3
이란             3
우즈베키스탄         2
몽고             2
슬로바키아          1
불가리아           1
스웨덴            1
크로아티아          1
뉴질랜드           1
남아프리카공화국       1
우루과이           1
에스토니아          1
이집트            1
아이슬란드          1
인도네시아          1
체코             1
말레이시아          1
Name: count, dtype: int64

## 5) 최종 데이터 프레임

In [59]:
movie_df

,VIEWNG_NMPR_CO,DISTB_CMPNY_NM,GENRE_NM,NLTY_NM,OPN_DE,DIST_CLEAN,DIST_MERGED,DIST_TOP30
0,4750104,(주)쇼박스,드라마,한국,2020-01-22,쇼박스,쇼박스,기타
1,4352669,(주)씨제이이엔엠,범죄,한국,2020-08-05,씨제이이엔엠,CJ ENM,CJ ENM
2,3812080,(주)넥스트엔터테인먼트월드(NEW),액션,한국,2020-07-15,넥스트엔터테인먼트월드,넥스트엔터테인먼트월드,넥스트엔터테인먼트월드
3,2406232,롯데컬처웍스(주)롯데엔터테인먼트,코미디,한국,2020-01-22,롯데컬처웍스롯데엔터테인먼트,롯데컬처웍스,롯데컬처웍스
4,1992214,워너브러더스 코리아(주),액션,미국,2020-08-26,워너브러더스 코리아,워너브러더스,워너브러더스
...,...,...,...,...,...,...,...,...
12467,445,메가박스중앙(주) 플러스엠 엔터테인먼트,범죄,한국,2025-04-16,메가박스중앙 플러스엠 엔터테인먼트,플러스엠,플러스엠
12468,430,(주)박수엔터테인먼트,애니메이션,미국,2025-04-10,박수엔터테인먼트,박수엔터테인먼트,기타
12469,424,(주)넥스트엔터테인먼트월드(NEW),드라마,일본,2023-11-29,넥스트엔터테인먼트월드,넥스트엔터테인먼트월드,넥스트엔터테인먼트월드
12470,405,씨제이 씨지브이(CJ CGV)(주),애니메이션,한국,2025-01-01,씨제이 씨지브이(CJ CGV),CJ CGV,CJ CGV


In [60]:
movie_df.info()

<class 'pandas.DataFrame'>
Index: 9917 entries, 0 to 12471
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   VIEWNG_NMPR_CO  9917 non-null   int64         
 1   DISTB_CMPNY_NM  9917 non-null   str           
 2   GENRE_NM        9917 non-null   str           
 3   NLTY_NM         9917 non-null   str           
 4   OPN_DE          9917 non-null   datetime64[us]
 5   DIST_CLEAN      9917 non-null   object        
 6   DIST_MERGED     9917 non-null   str           
 7   DIST_TOP30      9917 non-null   str           
dtypes: datetime64[us](1), int64(1), object(1), str(5)
memory usage: 697.3+ KB
